In [5]:
import pandas as pd

In [2]:
# import glob

In [5]:
# files = glob.glob("eniac-data-default/*.csv")
 
# df_list = []
# for filename in files:
#   data = pd.read_csv(filename)
#   df_list.append(data)
 
# df = pd.concat(df_list)

In [6]:
orders = pd.read_csv("eniac-data-default/orders.csv")
orderlines = pd.read_csv("eniac-data-default/orderlines.csv")
products = pd.read_csv("eniac-data-default/products.csv")
brands = pd.read_csv("eniac-data-default/brands.csv")

In [13]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

In [15]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


## orders pipeline

In [17]:
def start_pipeline(data): 
    return data.copy()

def set_types_orders(data): 
    return data.assign(created_date = pd.to_datetime(data['created_date']))

def remove_NaN_orders(data):
    return data.dropna()

In [31]:
orders.pipe(start_pipeline).pipe(set_types_orders).pipe(remove_NaN_orders).sample(5)

,order_id,created_date,total_paid,state
37187,336671,2017-03-19 09:15:17,115.97,Pending
58818,358356,2017-05-23 13:36:27,98.99,Pending
25514,324994,2017-02-16 00:09:56,64.97,Shopping Basket
161868,462128,2017-12-13 19:15:19,0.00,Shopping Basket
161028,461286,2017-12-13 00:15:08,734.00,Shopping Basket


In [25]:
orders.duplicated().value_counts()

False    226909
dtype: int64

## orderlines pipeline

In [40]:
orderlines.unit_price.sample(10)

135724       51.99
101607       82.99
108265       15.90
261491      158.99
182813      154.99
109766    2.304.60
42484        59.00
101527       30.92
241308      239.00
231227       64.99
Name: unit_price, dtype: object

In [64]:
def start_pipeline(data): 
    return data.copy()

def rename_id_o(data):
    return data.rename(columns={'id_order':'order_id'})

def set_types_orders(data): 
    return data.assign(date = pd.to_datetime(data['date']))

def remove_dots(data):
    return data.assign(price_nd = data['unit_price'].str.replace('\.','', regex=True))

def split_price(data):
    return (data
        .assign(euro = data['price_nd'].str[:-2])
        .assign(cent = data['price_nd'].str[-2:])
    )

def merge_price(data):
    return data.assign(price_per_unit = data['euro'] + '.' + data['cent'])

def set_type_price(data): 
    return data.assign(price_per_unit = pd.to_numeric(data['price_per_unit']))

def drop_unrelevant(data):
    return data.drop(columns = {'unit_price', 'price_nd', 'euro', 'cent', 'product_id'})

def rename_price(data):
    return data.rename(columns={'price_per_unit':'unit_price'})

In [65]:
(
orderlines
    .pipe(start_pipeline)
    .pipe(rename_id_o)
    .pipe(set_types_orders)
    .pipe(remove_dots)
    .pipe(split_price)
    .pipe(merge_price)
    .pipe(set_type_price)
    .pipe(drop_unrelevant)
    .pipe(rename_price)
    .sample(10)
)

,id,order_id,product_quantity,sku,date,unit_price
100266,1310615,385845,1,APP2450,2017-08-03 13:54:40,299.00
201127,1494717,462178,1,SAT0054,2017-12-13 20:05:18,79.99
164185,1441222,440936,1,SYN0171,2017-11-24 19:14:55,79.99
89995,1291559,376820,1,APP2075,2017-07-13 12:02:39,3099.00
124998,1367299,407274,1,WDT0358,2017-10-02 11:54:52,93.30
10200,1144653,308834,1,APP0698,2017-01-13 14:32:57,9.99
254295,1586624,500841,1,ZAG0032,2018-01-29 10:51:47,29.99
58923,1236872,350378,1,APP0927,2017-04-27 15:12:33,14.99
1555,1122312,300987,1,ELG0039,2017-01-02 20:54:44,46.99
275803,1621441,514896,2,OWC0024,2018-02-20 17:31:33,25.99


## products pipeline

In [56]:
products.duplicated(keep=False).value_counts()

False    9823
True     9503
dtype: int64

In [57]:
products[products.duplicated(keep=False)]

,sku,name,desc,price,promo_price,in_stock,type
100,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
101,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
102,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
103,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
104,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
...,...,...,...,...,...,...,...
18190,PAR0077,Parrot Bebop Drone 2 Power,Drone cuadricóptero quality camera integrated ...,699.9,6.733.892,0,11905404
18307,NKI0010,Nokia Wireless sphygmomanometer Plata,Sphygmomanometer for iPhone iPad and iPod App.,129.99,1.149.899,1,11905404
18308,NKI0010,Nokia Wireless sphygmomanometer Plata,Sphygmomanometer for iPhone iPad and iPod App.,129.99,1.149.899,1,11905404
18513,PLA0035,Bluetooth Headset Plantronics Explorer 80,egonómico wireless headset design for iPhone i...,39.99,349.896,1,5384


In [61]:
def start_pipeline(data): 
    return data.copy()

def drop_unrelevant(data):
    return data.drop(columns = {'in_stock', 'type'})


In [66]:
(
products
    .drop(start_pipeline)
    .drop(drop_unrelevant)
    .sample(10)

)

KeyError: '[<function start_pipeline at 0x000001445C658160>] not found in axis'

### from Joan:

In [ ]:
def common_orders(orders, orderlines): 
    return (orders
            .assign(order_check = orders['order_id'].isin(orderlines['id_order']))
            .query('order_check == True')
            ['order_id'].tolist())

def select_common_orders(data, orders_list: list):
    if 'order_id' in data.columns: 
        data.rename(columns={'order_id':'id_order'}, inplace=True)
    data_filtered = data.query('id_order == @orders_list')
    print(
        f"""
        Total {data_filtered.shape[0]} rows included over the original {data.shape[0]} rows.\n
        Kept the {round((data_filtered.shape[0] / data.shape[0])*100, 2)}% of the data.
        """)
    return data_filtered    

(
orders
    .pipe(start_pipeline)
    .pipe(set_types_orders)
    .dropna()
    .pipe(select_common_orders, orders_list=common_orders(orders, orderlines))
)